# Neo4J - with Python
##### Lukas Hinterleitner, Daniel Nepp
For a complete documentation visit: https://neo4j.com/developer/python/

---

## Installation
### 1. Download and install Neo4j Desktop:
https://neo4j.com/download/

Before you can download the installation file you have to provide your data.
If you don't want give your data away, you can type in random stuff :)

An activation key gets displayed while downloading (shown in the picture below).
Keep it, you will need it later during the installation.
<br><br>

![Neo4J Activation Key](screenshots/key.png)
<br><br>

After the installation finished you should see the home screen which is displayed in the image below.
We will get back to it later when we create our first Neo4J graph database.

![Home screen Neo4J Desktop](screenshots/home_screen.png)
<br><br>

### 2. Create a Conda-Environment :

`$ conda create --name neo4j python=3.8.2`


### 3. Activate Conda-Environment :

`$ conda activate neo4j`


### 4. Install the Neo4j driver :
https://anaconda.org/conda-forge/neo4j-python-driver

`$ conda install -c conda-forge neo4j-python-driver`

### 5. Install jupyter notebook : ##

`$ conda install -c anaconda jupyter`

or install jupyter lab

`$ conda install -c conda-forge jupyterlab`

or install jupyter in pycharm...


---

## Using Neo4j : ##

In [ ]:
from neo4j import GraphDatabase

uri, user, password = 'uri/', 'name', 'password'

def _create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    greeting = session.write_transaction(_create_and_return_greeting, "Hello World!")
    print(greeting)

driver.close()

... the above is a neo hello world example! :)
